# Exact solution

Ethier C.R., Steinman D.A., Exact fully 3d ?Nvier-Stokes solutions for banchmarking


We test here for a stationary solution (d_const = 0). Any mesh should work.

In [1]:
from dolfin import *
import numpy as np
import mshr as mshr

# Should stay 1 here (as we used normalized equation)
mu = Constant(1.)
muv = mu.values()[0]


baseR = 1.
topR = 1.5
h = 2.

outerCyl = mshr.Cylinder(Point([0.,0.,0.]),Point([0.,0.,h]),baseR,topR)

In [2]:
# We mark physical boundary and setup the space of finite elements 
mark = {"generic": 0,
        "wall": 1}

class Bondary_w(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary 

wall = Bondary_w()

#File("bord.pvd") << subdomains

degree = 2
elemf0 = FiniteElement('P-', cell='tetrahedron',form_degree=0, degree=degree)
elemf1 = FiniteElement('P-', cell='tetrahedron',form_degree=1, degree=degree)
elemf2 = FiniteElement('P-', cell='tetrahedron',form_degree=2, degree=degree)
elemf3 = FiniteElement('P-', cell='tetrahedron',form_degree=3, degree=degree)
elemH = FiniteElement('Real',cell='tetrahedron',degree=0)

TH = MixedElement([elemf1,elemf2,elemf3,elemH])

In [3]:
mesh=mshr.generate_mesh(outerCyl,15)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)
bcu_wall = DirichletBC(W.sub(1), Constant((0.,0.,0.)), subdomains, mark["wall"])
dbcs = [bcu_wall]

In [4]:
source = Expression('x[2]*x[2]*x[2]*x[2]*x[2]*x[2]*x[2]',degree=7)
w_old = Function(W)
(u1_o, u2_o, u3_o, uh_o) = split(w_old)
scale = assemble((u3_o-source)*dx)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [5]:
# We define physical parameters

nbitt = 10

#T = 8.0
#timestep = Constant(T/nbitt)
#print(timestep.values()[0])
timestep = Constant(0.01)

noslip = Constant((0.,0.,0.))
force = Expression(('0.','0.','7*x[2]*x[2]*x[2]*x[2]*x[2]*x[2]/scale'),degree=7,scale=scale)


In [6]:
# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
(u1, u2, u3, uh) = TrialFunctions(W)
(v1, v2, v3, vh) = TestFunctions(W)
n = FacetNormal(mesh)

# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
ah = u3*vh*dx + uh*v3*dx
a = a1 + a2 + a3 + ah

In [7]:
t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))

    L = dot(force,v2)*dx 
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    print("Norm curl {}".format(norm(w1.sub(0))))
    print("Norm velocity {}".format(norm(w1.sub(1))))
    print("Norm pressure {}".format(norm(w1.sub(2))))
    print("Norm harmonic pressure {}".format(norm(w1.sub(3))))

    w_old.assign(w1)
    t = t + timestep.values()[0]


85588
t = 0.000
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Norm curl 1.4845971348082344e-15
Norm velocity 8.733844684541822e-17
Norm pressure 0.4871853896594921
Norm harmonic pressure 8.419305348506379e-18
t = 0.010
Norm curl 1.588515309685295e-15
Norm velocity 1.147112170156423e-16
Norm pressure 0.48718538965949204
Norm harmonic pressure 8.444321036373302e-19
t = 0.020
Norm curl 1.6043740348512565e-15
Norm velocity 1.3896929979444833e-16
Norm pressure 0.48718538965949176
Norm harmonic pressure 6.939953634763208e-19
t = 0.030
Norm curl 1.7620953713834968e-15
Norm velocity 1.2384537634579826e-16
Norm pressure 0.4871853896594919
Norm harmonic pressure 1.2003352201490743e-18
t = 0.040
Norm curl 1.5247111980726596e-15
Norm velocity 9.622937988612201e-